In [1]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import Sequential
from keras import layers
from keras import backend as K
from keras.layers.core import Dense
from keras import regularizers
from keras.layers import Dropout
from keras.constraints import max_norm

Using TensorFlow backend.


In [2]:
# Import data
train = pd.read_csv('../../01_input/train.csv')
test = pd.read_csv('../../01_input/test.csv')

In [3]:
train.shape

(200000, 202)

In [4]:
test.shape

(200000, 201)

In [5]:
#Check num of cases in label 
print(train.target.value_counts())
print(train.target.value_counts()[1]/train.target.value_counts()[0])

0    179902
1     20098
Name: target, dtype: int64
0.1117163789174106


In [6]:
train_features = train.drop(['target', 'ID_code'], axis=1)
train_targets = train['target']
test_features = test.drop(['ID_code'], axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets, test_size = 0.25, random_state = 50)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
test_features = sc.transform(test_features)

In [9]:
# Add RUC metric to monitor NN
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [10]:
input_dim = X_train.shape[1]
input_dim

200

In [11]:
# Try early stopping
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

In [12]:
model = Sequential()
# Input layer
model.add(Dense(units=200, activation='relu', dtype='float32', name='Hidden1', input_shape=(200,),kernel_initializer = "normal", kernel_regularizer=regularizers.l2(0.005), 
                kernel_constraint = max_norm(5.)))
model.add(Dense(units=200, activation='relu', dtype='float32', name='Hidden2'))
model.add(Dense(units=150, activation='relu', dtype='float32', name='Hidden3'))
model.add(Dense(units=100, activation='relu', dtype='float32', name='Hidden4'))
model.add(Dense(units=150, activation='relu', dtype='float32', name='Hidden5'))
model.add(Dense(units=200, activation='linear', dtype='float32', name='Output'))
model.compile(loss='mean_squared_error', optimizer='SGD')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Hidden1 (Dense)              (None, 200)               40200     
_________________________________________________________________
Hidden2 (Dense)              (None, 200)               40200     
_________________________________________________________________
Hidden3 (Dense)              (None, 150)               30150     
_________________________________________________________________
Hidden4 (Dense)              (None, 100)               15100     
_________________________________________________________________
Hidden5 (Dense)              (None, 150)               15150     
_________________________________________________________________
Output (Dense)               (None, 200)               30200     
Total params: 171,000
Trainable params: 171,000
Non-trainable params: 

In [ ]:
model.fit(X_train, X_train, batch_size = 13384, epochs = 10000, validation_data = (X_train, X_train), callbacks = [callback])

Instructions for updating:
Use tf.cast instead.
Train on 150000 samples, validate on 150000 samples
Epoch 1/10000
150000/150000 [==============================] - 2s 15us/step - loss: 1.5129 - val_loss: 1.5118
Epoch 2/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.5109 - val_loss: 1.5098
Epoch 3/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.5090 - val_loss: 1.5079
Epoch 4/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.5072 - val_loss: 1.5061
Epoch 5/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.5054 - val_loss: 1.5044
Epoch 6/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.5036 - val_loss: 1.5026
Epoch 7/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.5019 - val_loss: 1.5010
Epoch 8/10000
150000/150000 [==============================] - 2s 14us/step - loss: 1.5003 - val_loss: 1.4993
Epoch 9/10000
150000

150000/150000 [==============================] - 2s 13us/step - loss: 1.4212 - val_loss: 1.4206
Epoch 73/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4202 - val_loss: 1.4196
Epoch 74/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4191 - val_loss: 1.4185
Epoch 75/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4181 - val_loss: 1.4175
Epoch 76/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4170 - val_loss: 1.4164
Epoch 77/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4160 - val_loss: 1.4154
Epoch 78/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4150 - val_loss: 1.4144
Epoch 79/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4139 - val_loss: 1.4133
Epoch 80/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.4129 - val_loss: 1.4123
Epoch 81/10000
1

150000/150000 [==============================] - 2s 13us/step - loss: 1.3516 - val_loss: 1.3511
Epoch 146/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3508 - val_loss: 1.3503
Epoch 147/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3499 - val_loss: 1.3494
Epoch 148/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3491 - val_loss: 1.3486
Epoch 149/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3482 - val_loss: 1.3477
Epoch 150/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3473 - val_loss: 1.3469
Epoch 151/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3465 - val_loss: 1.3460
Epoch 152/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3456 - val_loss: 1.3452
Epoch 153/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.3448 - val_loss: 1.3443
Epoch 15

150000/150000 [==============================] - 2s 13us/step - loss: 1.2941 - val_loss: 1.2937
Epoch 219/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2934 - val_loss: 1.2930
Epoch 220/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2927 - val_loss: 1.2923
Epoch 221/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2920 - val_loss: 1.2915
Epoch 222/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2912 - val_loss: 1.2908
Epoch 223/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2905 - val_loss: 1.2901
Epoch 224/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2898 - val_loss: 1.2894
Epoch 225/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2891 - val_loss: 1.2887
Epoch 226/10000
150000/150000 [==============================] - 2s 13us/step - loss: 1.2884 - val_loss: 1.2880
Epoch 22

In [ ]:
X_train

In [ ]:
x_ae=model.predict(X_train)
x_ae